In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import helpers 

## Q1 Make Trackbar for the following images to tune the parameters of canny detection.
_____________________________________

In [2]:
img = cv2.imread('solidWhiteCurve.jpg')
#print(type(img))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gray_blur = cv2.GaussianBlur(gray, (3,3), 0)

cv2.namedWindow('Canny Detection')

def nothing(x):
    pass

cv2.createTrackbar('Threshold 1','Canny Detection', 0, 100, nothing)
cv2.createTrackbar('Threshold 2','Canny Detection', 0, 200, nothing)

threshold1 = threshold2 = 0

while True:
    threshold1 = cv2.getTrackbarPos('Threshold 1','Canny Detection')
    threshold2 = cv2.getTrackbarPos('Threshold 2','Canny Detection')
    edges_image = cv2.Canny(gray_blur, threshold1, threshold2)

    cv2.imshow('Canny Detection', edges_image)
    
    k = cv2.waitKey(1)
    
    if k == ord('q'):
        break

cv2.destroyAllWindows()


# My observation is Trackbar is the best solution to know our pramters ... like in this situation when 
# we increase threshold 1 and 2 the edges will appear clear without any addition details

## Q2 The out of question 1 make it as input for question 2 after make region of interest then make trackbar for houghlines parameters (threshold ,min_line_length max_line_gap) and then draw 2 line on image.
____________________________________

In [3]:
def helpers_masked_edges(edges):
    mask = np.zeros_like(edges)
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(edges.shape) > 2:
        channel_count = edges.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    imshape = edges.shape        
    vertices = np.array([[((1/6*imshape[1]),imshape[0]),((5/12*imshape[1]), (3/5*imshape[0])), ((7/12*imshape[1]), (3/5*imshape[0])), ((9/10*imshape[1]),imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)
    return(masked_edges)

In [4]:
RHO = 1                
THETA = np.pi/180       
def hough_lines(img, MIN_VOTES, MIN_LINE_LEN, MAX_LINE_GAP):
    lines = cv2.HoughLinesP(img, RHO, THETA, MIN_VOTES, np.array([]), minLineLength=MIN_LINE_LEN, maxLineGap=MAX_LINE_GAP)
    return lines

In [5]:
def helpers_draw_lines(lines, masked_edges):
    color = [243, 105, 14]
    thickness = 12
    lines_image = np.zeros((masked_edges.shape[0], masked_edges.shape[1], 3), dtype=np.uint8)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(lines_image, (x1, y1), (x2, y2), color, thickness)
    return(lines_image)

In [6]:
def helpers_edges(gray,threshold1,threshold2):
    kernel_size = 5 #kernel size for Gaussian smoothing / blurring
    blur_gray= cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)#Gaussian smoothing / blurring

    edges = cv2.Canny(blur_gray, threshold1, threshold2)
    return(edges)

In [7]:
cv2.namedWindow('image')

cv2.createTrackbar('MIN_VOTES', 'image', 10, 50, nothing)   
cv2.createTrackbar('MIN_LINE_LEN', 'image', 1, 200, nothing)   
cv2.createTrackbar('MAX_LINE_GAP', 'image', 1, 200, nothing) 

while True:
    img = cv2.imread('solidWhiteCurve.jpg') 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    edges_img = helpers_edges(gray_img,50,64)
    ROI_edges_img = helpers_masked_edges(edges_img)
    
    min_votes = cv2.getTrackbarPos('MIN_VOTES','image')
    min_line_len = cv2.getTrackbarPos('MIN_LINE_LEN','image')
    max_line_gap = cv2.getTrackbarPos('MAX_LINE_GAP','image')
    
    lines = hough_lines(ROI_edges_img, min_votes, min_line_len, max_line_gap)
    hough_lines_image = helpers_draw_lines(lines, ROI_edges_img)
    
    cv2.imshow('image', hough_lines_image)
    
    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cv2.destroyAllWindows()


#My observation is , the the best parameters is Min_Votes = 15, MIN_LINE_LEN = 115, MAX_LINE_GAP = 34